In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import tensorflow as tf
import sciann as sci
import scipy.io 
import time
import random

---------------------- SCIANN 0.6.3.1 ---------------------- 
For details, check out our review paper and the documentation at: 
 +  "https://www.sciencedirect.com/science/article/pii/S0045782520307374", 
 +  "https://arxiv.org/abs/2005.08803", 
 +  "https://www.sciann.com". 

 Need support or would like to contribute, please join sciann`s slack group: 
 +  "https://join.slack.com/t/sciann/shared_invite/zt-ne1f5jlx-k_dY8RGo3ZreDXwz0f~CeA" 
 


Constantsb

In [2]:
sigma = 72.0;        # surface tension [mN/m]
grav = 1.0;     # gravitational acceleration [mm/s^2]
rneedle = 1.0;       # radius of the needle [mm]
deltarho = 1.0;   # density difference [10^6 kg/m^3]
p_l = 70.2799663375; # laplace pressure
s = 5.1947
eps = 1e-8

Repeatability

In [3]:
np.random.seed(42)
tf.random.set_seed(42)

Create 1-D Points

In [4]:
# define computational "grid"
s_train = np.arange(0,s,0.01, dtype='float64').reshape(-1,1)

# define s variable
s_var = sci.Variable("s",dtype='float64')

# define r and z functional with 8 layers of 20 Densely connected neurons, with tanh activation
r, z = sci.Functional(['r','z'], [s_var], 8 *[20], 'tanh')


In [14]:
from sciann.utils.math import diff, sign, sin, div, cos, atan
# setup the PDE model
dr_ds = diff(r, s_var)
d2r_d2s = diff(r, s_var, order=2)
dz_ds = diff(z, s_var)
d2z_ds2 = diff(z,s_var, order=2)

psi = atan(div(dz_ds, dr_ds))
kappa_s = - d2r_d2s / (1.0-(dr_ds)**2) ** 0.5
kappa_phi = 1 / r * dz_ds

L1 = sci.PDE(p_l - z - sigma*(kappa_s + kappa_phi))
L2 = dr_ds - cos(psi)
L3 = dz_ds - sin(psi)

r_bot_bc = (1-sign(r - eps)) *r 
r_top_bc = (1-sign(r - (1+eps))) * r
z_bc = (1-sign(z - eps)) *z
psi_bc = (1 - sci.utils.sign(r - (s+eps))*psi)

# The model is formed with input `s` and condition `c1`.

targets = [L1, L2, L3, r, z, psi]
target_vals = ['zeros', 'zeros', 'zeros', r_bc, z_bc, psi_bc]


model1 = sci.SciModel(
    [s_var], 
    targets)


In [21]:
hist1 = model1.train(
       [s_train],
       target_vals,
       epochs = 1000,
       learning_rate = 0.00005,
       verbose=1
       )

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [19]:
s_train.flatten().shape

(520,)